[View in Colaboratory](https://colab.research.google.com/github/hlecuanda/jupyter-notebooks-of-all-kinds/blob/master/DriveAPI.ipynb)

In [6]:
try:
    from oauth2client.service_account import ServiceAccountCredentials
    from google.oauth2 import service_account
except:
    !pip install google-auth google-auth-httplib2 google-api-python-client
    from oauth2client.service_account import ServiceAccountCredentials
finally:
    from apiclient import discovery
    import httplib2
    from google.colab import files
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))


SCOPES = ['https://www.googleapis.com/auth/drive',
          'https://www.googleapis.com/auth/drive.file',
          'https://www.googleapis.com/auth/drive.metadata.readonly']
SERVICE_ACCOUNT_FILE = 'MezaOps-9483d786f5ef.json'

credentials = ServiceAccountCredentials.from_json_keyfile_name(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)


delegated_credentials = credentials.create_delegated('hector@lecuanda.com')  
delegated_http = delegated_credentials.authorize(httplib2.Http())
drive  = discovery.build('drive', 'v3', http=delegated_http)


In [21]:
import IPython
js_code = '''
document.querySelector("#output-area").appendChild(document.createTextNode("hello world!"));
'''
display(IPython.display.Javascript(js_code))

<IPython.core.display.Javascript object>

In [20]:
#@title Directory id { run: "auto", vertical-output: true, output-height: 100, form-width: "50%", display-mode: "form" }
dir_id = "1kghdjPxiHRvQY2sQa1koli2csSGqni8_" #@param {type:"string"}
drivequery="'{}' in parents and mimeType = \
'application/vnd.google-apps.document'".format(dir_id)
docs = drive.files().list(q=drivequery).execute()

print("{} documents selected".format(len(docs['files'])))


55 documents selected


In [11]:
import pprint
pprint.pprint(docs['files'])

[{'id': '1ydYeSy9A5W6NzEM-w0AjuDyHo6ZaZsNPGfzzmU9cVDo',
  'kind': 'drive#file',
  'mimeType': 'application/vnd.google-apps.document',
  'name': '20141231-statements-x7399-.pdf'},
 {'id': '1l99Dvfyzx0UXbH-Cx_kINhhos6fRdaTtw3XMK0dH1DI',
  'kind': 'drive#file',
  'mimeType': 'application/vnd.google-apps.document',
  'name': 'Untitled document'},
 {'id': '1J2nq5ootxcaz5zlV80K_d8HRsvvz6zLg6z-cbOHt8aU',
  'kind': 'drive#file',
  'mimeType': 'application/vnd.google-apps.document',
  'name': '20161230-statements-x7399-.pdf'},
 {'id': '1dGmSJQYFekek0JUvMFhbv-r43pHdFEK4T2JfW_e3Ngg',
  'kind': 'drive#file',
  'mimeType': 'application/vnd.google-apps.document',
  'name': '20161130-statements-x7399-.pdf'},
 {'id': '1junwqRRBJWIsTlTYTQFTR5bU7BXpUYgebEpJykFp59E',
  'kind': 'drive#file',
  'mimeType': 'application/vnd.google-apps.document',
  'name': '20161031-statements-x7399-.pdf'},
 {'id': '1dKR4R5Rc775Js5MJP8VQhWg5zLXFnPNFAfSXY1jZSlM',
  'kind': 'drive#file',
  'mimeType': 'application/vnd.google-

In [5]:
# Search query reference:
# https://developers.google.com/drive/v2/web/search-parameters
listed = drive.ListFile({'q': "'%s' in parents and mimeType = 'application/pdf'" % dir_id }).GetList()
doc_files = drive.ListFile({'q': "'%s' in parents and mimeType = 'application/vnd.google-apps.document'" % dir_id }).GetList()

debug = False
files=[]
row="{},{}.{}"
if windows_fmt:
  row+='\n'
for file in doc_files:
  files.append(file['id'])
  if debug:
    print(row.format(file['title'], file['id'], file['mimeType']))

print("Total Docs: {}".format(len(doc_files)))
doc_files[0:3]
doc_files[len(doc_files)-3:len(doc_files)]

NameError: ignored